In [7]:
#################
## MYSQL STUFF ##
#################

In [8]:
import os
import mysql.connector
from mysql.connector import errorcode
from pathlib import Path
import json
import logging


currentWorkingDirectory = os.path.abspath('')
secretFile = json.load(open(currentWorkingDirectory+'/config/secrets.json'))
dbhost = secretFile['dbhost']
dbname = secretFile['db']
dbuser = secretFile['dbuser']
dbpass = secretFile['dbpass']


In [14]:
try:
    cnx = mysql.connector.connect(user=dbuser, password=dbpass,
                                  host=dbhost,
                                  database=dbname)
    print("connection to db successfully established.")

except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Invalid user credentials.")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
    cnx.close()

connection to db successfully established.


In [5]:
#
# insert a userid to usertable table example
#
def insert_user(uid):
    query = "INSERT INTO users VALUES (%s)"
    args = (uid, )

    try:
        cursor = cnx.cursor()
        cursor.execute(query, args)

        cnx.commit()
        
    except Exception as e:
        print("error: ", e)

    finally:
        cursor.close()
        
insert_user(12345)

error:  1062 (23000): Duplicate entry '12345' for key 'PRIMARY'


In [11]:
#
# check if userid already exists
#
def find_user(uid):
    query = "SELECT * FROM users WHERE uid=%s"
    args = (uid,)

    try:
        print("starting operation: find_user")
        cursor = cnx.cursor(buffered=True)
        cursor.execute(query, args)
        rows = cursor.fetchall()

        if len(rows) > 0:
            #if user already exists in db return False
            return True
        else:
            #if user does not exist in db return False
            return False

    except Exception as e:
        print(e)

    finally:
        cursor.close()


        
find_user(1234)
    



starting operation: find_user


False

In [36]:
def find_soundfile(soundfilename):

    query = "SELECT * FROM soundfiles WHERE file_name=%s"
    args = (soundfilename,)

    try:
        print("starting operation: find_soundfile")
        cursor = cnx.cursor(buffered=True)
        cursor.execute(query, args)
        rows = cursor.fetchall()

        if len(rows) > 0:
            #if user already exists in db return False
            soundfile_id = rows[0][0]
            return True
        else:
            #if user does not exist in db return False
            return False

    except Exception as e:
        print(e)

    finally:
        cursor.close()
        
        
find_soundfile('autism')

starting operation: find_soundfile


True

In [44]:
def get_soundfile_id(soundfilename):
    query = "SELECT * FROM soundfiles WHERE file_name=%s"
    args = (soundfilename,)

    try:
        print("starting operation: get_soundfile_id")
        cursor = cnx.cursor(buffered=True)
        cursor.execute(query, args)
        rows = cursor.fetchall()

        if len(rows) > 0:
            #if user already exists in db return False
            soundfile_id = rows[0][0]
            return soundfile_id
        else:
            #if user does not exist in db return False
            return -1

    except Exception as e:
        print(e)

    finally:
        cursor.close()
        
get_soundfile_id('autism')

starting operation: get_soundfile_id


2

In [61]:
def find_keyword_for_user(user_id, hotkeyword):
    query = "SELECT * FROM user_soundfiles WHERE user_id=%s AND soundkeyword=%s"
    args = (user_id, hotkeyword)

    try:
        print("starting operation: find_keyword_for_user")
        cursor = cnx.cursor(buffered=True)
        cursor.execute(query, args)
        rows = cursor.fetchall()

        if len(rows) > 0:
            #if user already exists in db return False
            return True
        else:
            #if user does not exist in db return False
            return False

    except Exception as e:
        print(e)

    finally:
        cursor.close()


        
find_keyword_for_user(286615413381529601, 'ho')

starting operation: find_keyword_for_user


True

In [63]:
def insert_user_soundfile(user_id, soundfile_id, soundkeyword):
    query = "INSERT INTO user_soundfiles VALUES (%s, %s, %s)"
    args = (user_id, soundfile_id, soundkeyword)

    try:
        cursor = cnx.cursor()
        cursor.execute(query, args)

        cnx.commit()
        
    except Exception as e:
        print("error: ", e)

    finally:
        cursor.close()
    
insert_user_soundfile(286615413381529601, 2, 'asmon1')

In [67]:
#
# insert new soundfile and hotkeyword to the user_soundfiles function
#

#realid = 286615413381529601
wrongid = 213442134213412423

#tbd: 
# check if user exists
# if user exists check if entered soundfile exists in soundfiles table
# if soundfile exists check if hotkeyword is not too long
# if all criteria is fullfilled add the row to the database

def addfavourite(userid, soundfilename, hotkeyword):
    
    userExists = find_user(userid)
    soundfileExists = find_soundfile(soundfilename)
    hotkeywordForUserExists = find_keyword_for_user(userid, hotkeyword)
    
    if userExists == False:
        print("user does not exist, please use signup first")
        
    elif soundfileExists == False:
        print("Pahie does not know this soundfile. Please make sure to enter an existing soundfile!")
        
    elif len(hotkeyword) >= 20:
        print("Please enter a shorter keyword!")
        
    elif hotkeywordForUserExists == True:
        print("You already used this hotkeyword! Please enter a different one.")
        
        
    else:
        user_id = userid
        soundfile_id = get_soundfile_id(soundfilename)
        hotkeyword
        print(user_id,",", soundfile_id, ",", hotkeyword)
        
        #insert row here!
        
        


        
addfavourite(wrongid, 'nissel-krimsekt', 'asmon2')

starting operation: find_user
starting operation: find_soundfile
starting operation: find_keyword_for_user
user does not exist, please use signup first


In [7]:
cnx.close()

In [4]:
######################
## REDDIT API STUFF ##
######################

In [5]:
import os
from pathlib import Path
import json
import logging
import praw

currentWorkingDirectory = os.path.abspath('')
secretFile = json.load(open(currentWorkingDirectory+'/config/secrets.json'))

use_script=secretFile['use_script']
client_secret=secretFile['client_secret']
user_agent=secretFile['user_agent']
username=secretFile['username']
password=secretFile['password']

In [6]:
reddit = praw.Reddit(client_id = use_script, 
                     client_secret = client_secret,
                     user_agent = user_agent,
                     username = username,
                     password = password)

In [7]:
subreddit_memes = reddit.subreddit("dankmemes")
hot_memes = subreddit_memes.hot(limit=5)

In [8]:
type(hot_memes)

praw.models.listing.generator.ListingGenerator

In [9]:
for i in hot_memes:
    print(i.title, i.url)

ASTRONOMIA COFFIN DANCE IS YOUR MARCH 2020 MEME OF THE MONTH https://i.imgur.com/X9SDUVN.gif
Did I miss anyone? https://i.redd.it/gkn1xu1cfcv41.gif
Watch where you're going https://i.redd.it/f0zkm19k9dv41.gif
The real battle royale https://i.redd.it/nvnb098z3cv41.jpg


In [1]:
######################
##### QUOTE API ######
######################

In [4]:
import requests
import json
url = ' https://opinionated-quotes-api.gigalixirapp.com/v1/quotes'
response = requests.get(url).json()

quote = response['quotes'][0]['quote']
author = response['quotes'][0]['author']
print(f"quote:{quote}\nauthor:{author}")

quote:The fact that one can lose one's sense of self in an ocean of tranquility does not mean that one's consciousness is immaterial or that it presided over the birth of the universe.
author:Sam Harris


In [5]:
######################
##### JOKE API #######
######################

In [64]:
jokeurl = 'https://sv443.net/jokeapi/v2/joke/Miscellaneous?blacklistFlags=racist,sexist'
payload = {}
headers = {'Accept':'application/json'}

response = requests.request("GET", jokeurl, headers=headers, data=payload).json()

In [65]:
response

{'category': 'Miscellaneous',
 'type': 'twopart',
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False},
 'setup': 'Two guys walked into a bar.',
 'delivery': 'The third guy ducked.',
 'id': 170,
 'error': False}

In [66]:
print(response['setup'], response['delivery'])

Two guys walked into a bar. The third guy ducked.


In [47]:
#############################
##### TRUMP QUOTE API #######
#############################

trumpurl = 'https://api.tronalddump.io/random/quote'
response = requests.get(trumpurl).json()
response['value']

'.@lindseygraham, who had zero in his presidential run before dropping out in disgrace- saying the most horrible things about me on Fox News.'

In [ ]:
#############################
##### FOAAS API #############
#############################

In [ ]:
#https://www.foaas.com/